## Подготовка данных


Выбранный текст - Человек, который принял жену за шляпу

In [217]:
#constants 
file_name = 'Человек,который_принял_жену_за_шляпу.pdf' # реальный текст романа с 14 по 112 страницы 
sentence_len = 100

In [218]:
! pip install pdfquery
! pip install pandas
! pip install --user -U nltk

import sys
!{sys.executable} -m pip install numpy

! pip install torch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new 

In [219]:
#imports 
from pdfquery import PDFQuery

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import nltk

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import string
import re

from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm

In [220]:
pdf = PDFQuery(file_name)
pdf.load(*range(13, 53)) #TODO("увеличить?")

text_elements = pdf.pq('LTTextLineHorizontal')

In [221]:
text = "".join([t.text for t in text_elements]).replace('\n', '')

In [222]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/esoboleva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [223]:
def tokenize_ru(file_text):
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)

    # let's delete punctuation symbols
    tokens = [i for i in tokens if (i not in string.punctuation)]

    # deleting other symbols
    punct = ['—', ',', '.', '...']
    tokens = [i for i in tokens if (i not in punct)]

    # cleaning words
    tokens = [re.sub("[a-z]","", re.sub("[0-9]","",
              i.replace("«", "")
              .replace("»", "")
              .replace(".", "")
              .replace("-", "")
              .replace('—', "")
              .replace(',', "")
              .replace('`', "")
                                        
              .lower()))
              for i in tokens]
    

    return tokens

In [224]:
sentences = [tokenize_ru(sent) for sent in sent_tokenize(text)]

In [225]:
alphabet = {'\n'}

loaded_text_path = 'text.txt'
text_file = open(loaded_text_path, "w")

for s in sentences:
    joined_s = " ".join(s)
    if len(joined_s) > sentence_len:
        text_file.write(joined_s)
        text_file.write('\n')
        # getting alphabet
        for c in joined_s:
            alphabet.add(c)
text_file.close()
alphabet = sorted(alphabet)
print(alphabet)
n_vocab = len(alphabet)

text = open("text.txt").read()
sentences = open("text.txt").readlines()
print('Symbols count: ', len(text))
print('Sentences count: ', len(sentences))

['\n', ' ', "'", 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']
Symbols count:  129586
Sentences count:  751


остались только кириллица и пробел - ура

сделаем one-hot encoding

In [226]:
int_to_vocab = {ii:word for ii, word in enumerate(alphabet)}
vocab_to_int = {word:ii for ii, word in int_to_vocab.items()}

def encode_text(text):
    int_text = [vocab_to_int[word] for word in text]

    one_hot_text = []
    for c in int_text:
        cur = np.zeros(n_vocab, dtype=int)
        cur[c] = 1
        one_hot_text.append(cur)
    return one_hot_text


def batch_data(text, sequence_length, batch_size):
    n_batches = len(text)//batch_size
    x, y = [], []
    text = text[:n_batches*batch_size]
    
    for ii in range(0, len(text)-sequence_length):
        i_end = ii+sequence_length        
        batch_x = text[ii:ii+sequence_length]
        x.append(batch_x)
        batch_y = text[i_end]
        y.append(batch_y)
    
    data = torch.from_numpy(np.asarray(x)), torch.from_numpy(np.asarray(y))
        
    return data

def to_loader(data, batch_size):
    return DataLoader(TensorDataset(*data), shuffle=True, batch_size=batch_size)


one_hot_text = encode_text(text)

## LSTM

In [227]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_vocab, hidden_size=256, num_layers=3, batch_first=True)
#         self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(256, n_vocab)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :] 
        # produce output
        x = self.linear(x)
        return x

In [228]:
n_epochs = 20
batch_size = 128
model = RNNModel()

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

train_loader = to_loader(batch_data(one_hot_text, sentence_len, batch_size), batch_size)

best_model = None
best_loss = np.inf

In [229]:
for epoch in range(n_epochs):
    model.train()
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(torch.float32)
        y_pred = model(X_batch)
        
        y_pred = y_pred.to(torch.float32)
        y_batch = y_batch.to(torch.float32)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(torch.float32)
            y_pred = model(X_batch)
            y_pred = y_pred.to(torch.float32)
            y_batch = y_batch.to(torch.float32)
            loss = loss_fn(y_pred, y_batch)
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))

Epoch 0: Cross-entropy: 2.4539
Epoch 1: Cross-entropy: 1.7686
Epoch 2: Cross-entropy: 1.6093
Epoch 3: Cross-entropy: 1.1163
Epoch 4: Cross-entropy: 1.3546
Epoch 5: Cross-entropy: 1.3182
Epoch 6: Cross-entropy: 1.2285
Epoch 7: Cross-entropy: 0.8202
Epoch 8: Cross-entropy: 0.9289
Epoch 9: Cross-entropy: 0.6384
Epoch 10: Cross-entropy: 0.5816
Epoch 11: Cross-entropy: 0.7738
Epoch 12: Cross-entropy: 0.8788
Epoch 13: Cross-entropy: 0.6497
Epoch 14: Cross-entropy: 0.6951
Epoch 15: Cross-entropy: 0.7154
Epoch 16: Cross-entropy: 0.9325
Epoch 17: Cross-entropy: 0.4460
Epoch 18: Cross-entropy: 0.3368
Epoch 19: Cross-entropy: 0.1637


In [230]:
torch.save(best_model, "model.pt")

In [231]:
file = open('text.txt', 'r')
lines = file.readlines()

In [232]:
model = RNNModel()
model.load_state_dict(torch.load("model.pt"))

<All keys matched successfully>

In [233]:
prompt_len = 20
sentence = np.random.randint(len(lines))
prompt = lines[sentence][0:prompt_len]

one_hot_prompt = encode_text(prompt)

In [234]:
print("Prompt: ", prompt)
print(prompt, end="")

with torch.no_grad():
    for i in range(500):
        
        one_hot_prompt = encode_text(prompt)
        x, y = batch_data(one_hot_prompt, prompt_len - 1, 1)
        x = x.to(torch.float32)
        
        prediction = model(x)
        index = int(prediction.argmax())

        result = int_to_vocab[index]
        print(result, end="")

        prompt = prompt + result
        prompt = prompt[1:]

print("\nDone.")

Prompt:  столь абсурдные ошиб
столь абсурдные ошиббкио тсоктуо тсиятл юи  впеорбее  вч еуг одвнеяс нвачнноый  рпаантиемм иин упзоаввлеянсык  ио драумсетв яи твроотн оеб оэднуаж иул едгуо  им пдрииммиастьию ткаан ии  всоявлшие  дсоятхеют  дионтаонваалщье  ив  сттаантиив аквтирке   сввеетшсея  тноувлеен ск осбоелм иил ивзеалт н апвиатлиас ьи  сслиабтеу твуодсак  силн овбоелмие  ввиедмуу  коагнроамтиартноомтиалщиях  прооббрааттиех  аонтоанлиий  пкроимхе тврооа  ии  псоибзоилтин овблаодтаи  птроидзеанттеийн 
вколрлеяв  еящ еищ еищ еиг реезл обтнаоя
Done.


## Марковская цепь


In [235]:
n_chars_to_consider = 6
transitions = dict()

In [236]:
for s in lines:
    prev = None
    i = 0
    while i < len(s):
        if prev == None:
            prev = s[:n_chars_to_consider]
            i += n_chars_to_consider
            continue
        new_state = prev[1:] + s[i]
        key = prev + '->' + new_state
        if not (key in transitions):
            transitions[key] = 1
        else:
            transitions[key] += 1
        prev = new_state
        i+=1

# нормируем
summ = 0
for t in transitions.values():
    summ += t

transitions = {k: v / summ for k, v in transitions.items()}
print("Transitions count: ", len(transitions.items()))

Transitions count:  81375


In [237]:
prompt_len = 11 # > n_chars_to_consider

sentence = np.random.randint(len(lines))
prompt = lines[sentence][0:prompt_len]

In [238]:
# предсказание
import operator

def our_transition(pair, suf):
    k, v = pair 
    return k.startswith(suf)
        

def predict(cur_line):
    suf = cur_line[len(cur_line) - n_chars_to_consider:] 
    trs = dict()
    for t in transitions.items():
        k, v = t
        if our_transition(t, suf):
            trs[k] = v
    if len(trs) == 0:
        return None, True
    
    max_val = max(trs.values())
    max_prob_key = max(trs, key = trs.get)
    
    next_char = max_prob_key[len(max_prob_key) - 1] #last 
    return cur_line + next_char, False

print("Real sentence:\n", lines[sentence])
print('Prompt: "%s"' % prompt)
cur = prompt
for i in range(100):
    cur, ended = predict(cur)
    if ended:
        print('Достигли перехода в конец строки, всё')
        break
    print(cur)


Real sentence:
 однако и здесь возможности эмили д были ограничены поскольку зрение ее быстро ухудшалось изза злокачественной глаукомы

Prompt: "однако и зд"
однако и зде
однако и здес
однако и здесь
однако и здесь 
однако и здесь в
однако и здесь ви
однако и здесь вит
однако и здесь витг
однако и здесь витге
однако и здесь витген
однако и здесь витгенш
однако и здесь витгеншт
однако и здесь витгенште
однако и здесь витгенштей
однако и здесь витгенштейн
однако и здесь витгенштейна
однако и здесь витгенштейна 
однако и здесь витгенштейна и
однако и здесь витгенштейна и 
однако и здесь витгенштейна и д
однако и здесь витгенштейна и др
однако и здесь витгенштейна и дру
однако и здесь витгенштейна и друг
однако и здесь витгенштейна и други
однако и здесь витгенштейна и других
однако и здесь витгенштейна и других 
однако и здесь витгенштейна и других н
однако и здесь витгенштейна и других не
однако и здесь витгенштейна и других нез
однако и здесь витгенштейна и других неза
однако и здесь ви